# Assignment: Credit Card Fraud Detection revised

we return to the credit card use case (last time we used Random Forests).


## Pre-Stage
Use your pre-processing from the last assignment as input.


In [1]:
#download data
#get the data
!git clone https://github.com/keuperj/DATA.git

Cloning into 'DATA'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 126 (delta 11), reused 39 (delta 11), pack-reused 87
Receiving objects: 100% (126/126), 185.56 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (86/86), done.


In [2]:
!unzip DATA/creditcard.csv.zip


Archive:  DATA/creditcard.csv.zip
  inflating: creditcard.csv          


In [25]:
import pandas as pd
data = pd.read_csv("creditcard.csv")
len(data)

284807

In [46]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix


In [50]:
# your preprocessing code
data.fillna(0, inplace=True) #replace NaN with zeros
data.replace([np.inf, -np.inf], 0, inplace=True) #also replace of inf values

data2 = data.copy()
data2 = data2[len(data)//2:]

x_labels_train = data2["Class"]

data2.drop(columns = ["Class"])
X_train=data2.to_numpy(dtype='float') #data type float

data3 = data.copy()
data3 = data3[:len(data)//2]

x_labels_test = data3["Class"]

data3.drop(columns = ["Class"])
X_test=data3.to_numpy(dtype='float') #data type float

#normalize data
from sklearn.preprocessing import normalize
X_train=normalize(X_train, axis=0)#normalize each variable
X_test=normalize(X_test, axis=0)#normalize each variable


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
142403,84692.0,1.125592,-0.147999,1.151008,1.158851,-0.823914,0.298829,-0.708732,0.294683,0.750838,...,-0.037453,0.024388,-0.052720,-0.003615,0.405682,-0.404597,0.063357,0.024091,12.99,0
142404,84693.0,-0.439952,0.683758,1.225814,0.639113,0.716765,0.089295,0.657718,0.034213,-0.472679,...,0.127443,0.588590,-0.145728,-0.322900,-0.299744,-0.310295,0.248317,-0.011493,19.00,0
142405,84694.0,-4.868108,1.264420,-5.167885,3.193648,-3.045621,-2.096166,-6.445610,2.422536,-3.214055,...,1.269205,0.057657,0.629307,-0.168432,0.443744,0.276539,1.441274,-0.127944,12.31,1
142406,84695.0,1.013114,-0.334412,1.305208,0.837406,-1.126833,-0.064321,-0.594753,0.147737,0.536360,...,0.017079,0.112210,-0.016084,0.595033,0.201073,0.278215,0.007457,0.030762,66.60,0
142407,84695.0,0.969231,-0.233554,0.238473,0.145793,-0.545741,-0.970680,0.347393,-0.209522,-0.342571,...,-0.362820,-1.417272,0.162136,0.541628,-0.079465,0.268702,-0.101237,0.028234,141.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


# Task: Train a MLP-Network
* Train and evaluate the model
* Compare the results to the Random Forrest
* Tune the hyper-parameters (like number of layers, neurons per layer, learning-rate and number of epochs ) to optimize the results

See: https://keras.io/api/

In [51]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

In [52]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
model.fit(X_train, x_labels_train, epochs=10)

Epoch 1/10
4451/4451 [==============================] - 15s 3ms/step - loss: 0.0105 - accuracy: 0.9993
Epoch 2/10
4451/4451 [==============================] - 14s 3ms/step - loss: 5.4551e-05 - accuracy: 1.0000
Epoch 3/10
4451/4451 [==============================] - 14s 3ms/step - loss: 2.8126e-06 - accuracy: 1.0000
Epoch 4/10
4451/4451 [==============================] - 14s 3ms/step - loss: 6.1267e-07 - accuracy: 1.0000
Epoch 5/10
4451/4451 [==============================] - 19s 4ms/step - loss: 3.8578e-08 - accuracy: 1.0000
Epoch 6/10
4451/4451 [==============================] - 14s 3ms/step - loss: 1.1989e-08 - accuracy: 1.0000
Epoch 7/10
4451/4451 [==============================] - 14s 3ms/step - loss: 1.3824e-06 - accuracy: 1.0000
Epoch 8/10
4451/4451 [==============================] - 14s 3ms/step - loss: 7.3923e-08 - accuracy: 1.0000
Epoch 9/10
4451/4451 [==============================] - 14s 3ms/step - loss: 2.1514e-09 - accuracy: 1.0000
Epoch 10/10
4451/4451 [==================

In [54]:
test_loss, test_acc = model.evaluate(X_test,  x_labels_test, verbose=2)

print('\nTest accuracy:', test_acc)

4451/4451 - 9s - loss: 0.0017 - accuracy: 0.9998 - 9s/epoch - 2ms/step

Test accuracy: 0.9998103976249695


In [55]:
predictions = model.predict(X_test)

4451/4451 [==============================] - 7s 2ms/step


In [56]:
i = 10
np.argmax(predictions[i])

0

In [57]:
x_labels_test[i]

0

In [58]:
confusion_matrix(x_labels_test, predictions)

ValueError: ignored